In [1]:
import torch
import transformers
import onnx
import numpy as np
import onnxruntime.training.onnxblock as onnxblock
from onnxruntime.training import artifacts

/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feature_extractor = transformers.MobileViTFeatureExtractor.from_pretrained("apple/mobilevit-xx-small")
model = transformers.MobileViTForImageClassification.from_pretrained("apple/mobilevit-xx-small")

/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/feature_extraction_mobilevit.py:28: FutureWarning: The class MobileViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileViTImageProcessor instead.
  warnings.warn(


In [3]:
model.classifier = torch.nn.Linear(320, 7)

onnx_name = "mobilevit_init_test.onnx"

# generates random pixel values for 5 images
# originally images were 256 by 256
random_input = {"pixel_values": torch.rand(5, 3, 512, 512)}

torch.onnx.export(model, random_input, onnx_name,
                    input_names=["input"], output_names=["output"],
                    export_params=True,
                    dynamic_axes={
                        "input": {0: "batch_size"},
                        "output": {0: "batch_size"}
                    },
                    do_constant_folding=False,
                    training=torch.onnx.TrainingMode.TRAINING)

/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/modeling_mobilevit.py:452: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  new_height = int(math.ceil(orig_height / patch_height) * patch_height)
/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/modeling_mobilevit.py:453: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  new_width = int(math.ceil(orig_width / patch_width) * patch_width)
/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/modeling_mo

In [4]:
class MobileViTWithLoss(onnxblock.TrainingModel):
    def __init__(self):
        super().__init__()
        self.loss = onnxblock.loss.CrossEntropyLoss()

    def build(self, loss_node_input_name):
        return self.loss(loss_node_input_name)

onnx_model = onnx.load(onnx_name)

eval_model = None
optimizer_model = None

training_block = MobileViTWithLoss()

artifacts.generate_artifacts(
    training_block,
    optimizer=artifacts.OptimType.AdamW
)

AttributeError: module 'onnxruntime.training.onnxblock' has no attribute 'TrainingModel'

In [2]:
from transformers import MobileViTFeatureExtractor, MobileViTForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = MobileViTFeatureExtractor.from_pretrained("apple/mobilevit-xx-small")
model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-xx-small")

inputs = feature_extractor(images=image, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits
outputs.loss

/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/feature_extraction_mobilevit.py:28: FutureWarning: The class MobileViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileViTImageProcessor instead.
  warnings.warn(


In [8]:

inputs.pixel_values.shape

torch.Size([1, 3, 256, 256])

In [18]:
model.classifier = torch.nn.Linear(320, 7)